# Receipt Item Extraction
Below is a simple example of using Hugging Face's serverless Inferece API on the following model: https://huggingface.co/AdamCodd/donut-receipts-extract

This can be hosted ourselves for processing reciept's into structure item data.  Alternative additional work can be done to try to inference the model on device with lower quantization.  Additional training pipelines could also be set up to improve the model over time base off customer receipt submissions or alternative data sources.

In [9]:
import requests

API_URL = "https://api-inference.huggingface.co/models/AdamCodd/donut-receipts-extract"
headers = {"Authorization": "Bearer <TOKEN_HERE>"}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query("coffee-chain-receipt-sample.jpg")

In [10]:
output

[{'generated_text': '<s_receipt><s_store_name> Coffee House</s_store_name><s_store_addr> 325 W 49th Austin, TX</s_store_addr><s_phone> (333)321-3333</s_phone><s_date> 07/17/2017</s_date><s_time> 09:13 AM</s_time><s_subtotal> $4.78</s_subtotal><s_svc></s_svc><s_tax> $0.00</s_tax><s_total> $4.78</s_total><s_tips></s_tips><s_discount></s_discount><s_line_items><s_item_key></s_item_key><s_item_name> Americano</s_item_name><s_item_value> $3.78</s_item_value><s_item_quantity> 1</s_item_quantity><sep/><s_item_key></s_item_key><s_item_name> Steamed Milk</s_item_name><s_item_value> $1.00</s_item_value><s_item_quantity> 1</s_item_quantity></s_line_items>'}]

In [1]:
# Caching to avoid having to inference sample receipt again
output = [{'generated_text': '<s_receipt><s_store_name> Coffee House</s_store_name><s_store_addr> 325 W 49th Austin, TX</s_store_addr><s_phone> (333)321-3333</s_phone><s_date> 07/17/2017</s_date><s_time> 09:13 AM</s_time><s_subtotal> $4.78</s_subtotal><s_svc></s_svc><s_tax> $0.00</s_tax><s_total> $4.78</s_total><s_tips></s_tips><s_discount></s_discount><s_line_items><s_item_key></s_item_key><s_item_name> Americano</s_item_name><s_item_value> $3.78</s_item_value><s_item_quantity> 1</s_item_quantity><sep/><s_item_key></s_item_key><s_item_name> Steamed Milk</s_item_name><s_item_value> $1.00</s_item_value><s_item_quantity> 1</s_item_quantity></s_line_items>'}]


In [4]:
from transformers import DonutProcessor
import re
processor = DonutProcessor.from_pretrained("AdamCodd/donut-receipts-extract")

# Decode generated output
decoded_text = output[0]['generated_text']
decoded_text = decoded_text.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
decoded_text = re.sub(r"<.*?>", "", decoded_text, count=1).strip()  # remove first task start token
decoded_text = processor.token2json(decoded_text)

/Users/dsharp/Development/coffee_run/python/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
decoded_text

{'store_name': 'Coffee House',
 'store_addr': '325 W 49th Austin, TX',
 'phone': '(333)321-3333',
 'date': '07/17/2017',
 'time': '09:13 AM',
 'subtotal': '$4.78',
 'svc': '',
 'tax': '$0.00',
 'total': '$4.78',
 'tips': '',
 'discount': '',
 'line_items': [{'item_key': '',
   'item_name': 'Americano',
   'item_value': '$3.78',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'Steamed Milk',
   'item_value': '$1.00',
   'item_quantity': '1'}]}

In [6]:
# Can use this to auto populate in flutter app
decoded_text['line_items']

[{'item_key': '',
  'item_name': 'Americano',
  'item_value': '$3.78',
  'item_quantity': '1'},
 {'item_key': '',
  'item_name': 'Steamed Milk',
  'item_value': '$1.00',
  'item_quantity': '1'}]

In [7]:
!pip freeze > requirements.txt